In [22]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

import sys
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="0" #! specify gpu here

import matplotlib
import matplotlib.pyplot as plt

import json
from pathlib import Path

import cv2

# from scipy import ndimage
import natsort
import numpy as np
from tqdm import tqdm
from rich import print

from vision_pipeline.llm_data_generator.labelme_importer import LabelMeImporter

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [26]:
labelme_importer = LabelMeImporter()
# load list of images, with ground truths, labelme style
# convert ground truths to Detections

dataset_dir = Path(os.path.expanduser("~/datasets2/reconcycle/2023-05-23_synthetic_dataset/output"))

image_folder = "images"
labelme_folder = "labelme"

labelme_dir = dataset_dir / labelme_folder
images_dir = dataset_dir / image_folder

json_paths = list(labelme_dir.glob('*.json'))
json_paths = natsort.os_sorted(json_paths)

image_paths = list(images_dir.glob('*.png')) + list(images_dir.glob('*.jpg')) 
image_paths = natsort.os_sorted(image_paths)

llm_output_list = []

tqdm_json_paths = tqdm(json_paths)
for idx, json_path in enumerate(tqdm_json_paths):
    tqdm_json_paths.set_description(f"{Path(json_path).stem}")

    

    json_data = json.load(open(json_path))
    filename = json_path.stem

    img_matches = [_img_path for _img_path in image_paths if filename == _img_path.stem.split('_')[0] ]

    colour_img_path = None
    if len(img_matches) > 0:
        for img_match in img_matches:
            if "depth" not in img_match.stem:
                colour_img_path = img_match                
                break
    
    print("json_path", json_path)
    print("colour_img_path", colour_img_path)

    colour_img = cv2.imread(str(colour_img_path))

    detections, graph_relations = labelme_importer.labelme_to_detections(json_data, colour_img)

    graph_relations_text = graph_relations.to_text()

    # print("detections", detections)
    print("graph relations:")
    print(graph_relations_text)

    llm_item = {
        "id": idx,
        "image": str(colour_img_path.relative_to(dataset_dir)),
        "conversations": [
            {
                "from": "human",
                "value": "<image>\nWhat are the objects and relations you see in this image?"
            },
            {
                "from": "gpt",
                "value": graph_relations_text
            },
        ]
    }
    llm_output_list.append(llm_item)

    # if idx > 2:
    #     break #! debug


# TODO: save to json file
print("llm_output_list", llm_output_list)

with open(dataset_dir / "llm_data.json", "w") as fp:
    json.dump(llm_output_list , fp, indent=4)

# TODO: I think we should use smaller images, with the device taking up most of the image. Because with CLIP the image will be scaled down a lot


00000001:   0%|          | 0/100 [00:00<?, ?it/s]

json_path /root/datasets2/reconcycle/2023-05-23_synthetic_dataset/output/labelme/00000001.json

colour_img_path /root/datasets2/reconcycle/2023-05-23_synthetic_dataset/output/images/00000001.jpg

graph relations:

Device 1: 
pcb in hca_back
battery in hca_back
pcb next to battery
battery next to pcb

Device 2: 
firealarm_back

...

colour_img_path /root/datasets2/reconcycle/2023-05-23_synthetic_dataset/output/images/00000100.jpg

graph relations:

Device 1: 
plastic_clip in hca_back
pcb_covered in hca_back
plastic_clip next to pcb_covered
pcb_covered next to plastic_clip

00000100: 100%|██████████| 100/100 [00:02<00:00, 40.31it/s]


llm_output_list
[
    {
        'id': 0,
        'image': 'images/00000001.jpg',
        'conversations': [
            {'from': 'human', 'value': '<image>\nWhat are the objects and relations you see in this image?'},
            {
                'from': 'gpt',
                'value': 'Device 1: \npcb in hca_back\nbattery in hca_back\npcb next to battery\nbattery next to 
pcb\n\nDevice 2: \nfirealarm_back'
            }
        ]
    },
    {
        'id': 1,
        'image': 'images/00000002.jpg',
        'conversations': [
            {'from': 'human', 'value': '<image>\nWhat are the objects and relations you see in this image?'},
            {'from': 'gpt', 'value': 'Device 1: \nfirealarm_back'}
        ]
    },
    {
        'id': 2,
        'image': 'images/00000003.jpg',
        'conversations': [
            {'from': 'human', 'value': '<image>\nWhat are the objects and relations you see in this image?'},
            {
                'from': 'gpt',
                'value': 'Device 1: \npcb in hca_back\nbattery in hca_back\npcb next to battery\nbattery next to 
pcb\n\nDevice 2: \npcb in internals\n'
            }
        ]
    },
    {
        'id': 3,
        'image': 'images/00000004.jpg',
        'conversations': [
            {'from': 'human', 'value': '<image>\nWhat are the objects and relations you see in this image?'},
            {
                'from': 'gpt',
                'value': 'Device 1: \npcb in internals\ninternals next to battery\nbattery next to 
internals\n\nDevice 2: \npcb in internals\n'
            }
        ]
    },
    {
        'id': 4,
        'image': 'images/00000005.jpg',
        'conversations': [
            {'from': 'human', 'value': '<image>\nWhat are the objects and relations you see in this image?'},
            {'from': 'gpt', 'value': 'Device 1: \npcb in hca_back\nbattery in hca_back\nbattery in pcb\n'}
        ]
    },
    {
        'id': 5,
        'image': 'images/00000006.jpg',
        'conversations': [
            {'from': 'human', 'value': '<image>\nWhat are the objects and relations you see in this image?'},
            {
                'from': 'gpt',
                'value': 'Device 1: \nplastic_clip in hca_back\ninternals in hca_back\nplastic_clip next to 
internals\ninternals next to plastic_clip\n\nDevice 2: \nfirealarm_front'
            }
        ]
    },
    {
        'id': 6,
        'image': 'images/00000007.jpg',
        'conversations': [
            {'from': 'human', 'value': '<image>\nWhat are the objects and relations you see in this image?'},
            {'from': 'gpt', 'value': 'Device 1: \npcb_covered in hca_back\n'}
        ]
    },
    {
        'id': 7,
        'image': 'images/00000008.jpg',
        'conversations': [
            {'from': 'human', 'value': '<image>\nWhat are the objects and relations you see in this image?'},
            {'from': 'gpt', 'value': 'Device 1: \ninternals in hca_back\n'}
        ]
    },
    {
        'id': 8,
        'image': 'images/00000009.jpg',
        'conversations': [
            {'from': 'human', 'value': '<image>\nWhat are the objects and relations you see in this image?'},
            {'from': 'gpt', 'value': 'Device 1: \npcb in internals\n'}
        ]
    },
    {
        'id': 9,
        'image': 'images/00000010.jpg',
        'conversations': [
            {'from': 'human', 'value': '<image>\nWhat are the objects and relations you see in this image?'},
            {
                'from': 'gpt',
                'value': 'Device 1: \npcb_covered in hca_back\n\nDevice 2: \npcb_covered in hca_back\n'
            }
        ]
    },
    {
        'id': 10,
        'image': 'images/00000011.jpg',
        'conversations': [
            {'from': 'human', 'value': '<image>\nWhat are the objects and relations you see in this image?'},
            {'from': 'gpt', 'value': 'Device 1: \nbattery_covered in firealarm_back\n'}
        ]
    },
    {
        'id': 11,
        'image': 'images/00000012.jpg',
        'conve